<h1>Extracting and Visualizing Stock Data</h1>

Extracting essential data from a dataset and displaying it is a necessary part of data science; therefore individuals can make correct decisions based on the data.


In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

<h3>Use yfinance to Extract Stock Data</h3>


In [3]:
tesla = yf.Ticker("TSLA")

In [4]:
tesla_data = tesla.history(period="max")
tesla_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29,1.266667,1.666667,1.169333,1.592667,281494500,0,0.0
2010-06-30,1.719333,2.028000,1.553333,1.588667,257806500,0,0.0
2010-07-01,1.666667,1.728000,1.351333,1.464000,123282000,0,0.0
2010-07-02,1.533333,1.540000,1.247333,1.280000,77097000,0,0.0
2010-07-06,1.333333,1.333333,1.055333,1.074000,103003500,0,0.0


In [5]:
tesla_data.reset_index(inplace=True)

<h3>Use Webscraping to Extract Tesla Revenue Data</h3>


In [6]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
data = requests.get(url).text

Parse the html data using `beautiful_soup`.


In [7]:
soup = BeautifulSoup(data, "html5lib")

In [8]:
read_html_tesla_data = pd.read_html(url)
tesla_revenue = read_html_tesla_data[1]
tesla_revenue.rename(columns={"Tesla Quarterly Revenue(Millions of US $)":"Date","Tesla Quarterly Revenue(Millions of US $).1":"Revenue"}, inplace=True )

In [9]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"", regex=True)

remove an null or empty strings in the Revenue column.


In [10]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [11]:
tesla_revenue.tail(10)

,Date,Revenue
43,2011-12-31,39
44,2011-09-30,58
45,2011-06-30,58
46,2011-03-31,49
47,2010-12-31,36
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


<h3>Use yfinance to Extract Stock Data</h3>


In [12]:
GameStop = yf.Ticker("GME")

In [13]:
gme_data = GameStop.history("max")

In [14]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


<h3>Use Webscraping to Extract GME Revenue Data</h3>


In [15]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

html_data = requests.get(url).text

In [16]:
soup = BeautifulSoup(html_data, "html5lib")

In [17]:
read_html_gme_data = pd.read_html(url)
gme_revenue = read_html_gme_data[1]

gme_revenue.rename(columns={"GameStop Quarterly Revenue(Millions of US $)":"Date",
                            "GameStop Quarterly Revenue(Millions of US $).1":"Revenue"}, inplace=True)
gme_revenue.head(10)

,Date,Revenue
0,2020-04-30,"$1,021"
1,2020-01-31,"$2,194"
2,2019-10-31,"$1,439"
3,2019-07-31,"$1,286"
4,2019-04-30,"$1,548"
5,2019-01-31,"$3,063"
6,2018-10-31,"$1,935"
7,2018-07-31,"$1,501"
8,2018-04-30,"$1,786"
9,2018-01-31,"$2,825"


In [18]:
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"", regex=True)

gme_revenue.tail(10)

,Date,Revenue
52,2007-04-30,1279
53,2007-01-31,2304
54,2006-10-31,1012
55,2006-07-31,963
56,2006-04-30,1040
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


<h3>Plot Stock Graphs</h3>


In [19]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = 1)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [20]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [21]:
make_graph(gme_data, gme_revenue, 'GameStop')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed